# CONVOLUTIONAL NEURAL NETWORK
## FOR FACE RECOGNITION

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
print ("Packages loaded")

Packages loaded


# LOAD DATA

In [2]:
# LOAD DATA
cwd = os.getcwd()
loadpath = cwd + "/data.npz"
l = np.load(loadpath)
print (l.files)

# PARSE LOADED DATA
trainimg   = l['trainimg']
trainlabel = l['trainlabel']
testimg    = l['testimg']
testlabel  = l['testlabel']
imgsize    = l['imgsize']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d TRAIN IMAGES" % (ntrain))
print ("%d TEST IMAGES" % (ntest))
print ("%d DIMENSIONAL INPUT" % (dim))
print ("%d CLASSES" % (nclass))

['trainimg', 'imgsize', 'testlabel', 'trainlabel', 'testimg']
544 TRAIN IMAGES
137 TEST IMAGES
6912 DIMENSIONAL INPUT
4 CLASSES


# DEFINE MODEL

In [3]:
tf.set_random_seed(0)

# MULTILAYER PERCEPTRON MODEL
n_input    = dim    # INPUT LAYER DIMENSION
n_filter1  = 64     # NUMBER OF FILTERS IN CONV1 
n_filter2  = 128    # NUMBER OF FILTERS IN CONV2
n_hid      = 256    # NUMBER OF HIDDEN LAYER NODES
n_output   = nclass # OUTPUT LAYER DIMENSION
x = tf.placeholder("float", [None, dim]) 
y = tf.placeholder("float", [None, nclass]) 

std = 0.1
W = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, n_filter1], stddev=std)),
    'wc2': tf.Variable(tf.random_normal([5, 5, n_filter1, n_filter2], stddev=std)),
    'wd1': tf.Variable(tf.random_normal(
            [(int)(imgsize[0]/4*imgsize[1]/4)*n_filter2, n_hid], stddev=std)),
    'wd2': tf.Variable(tf.random_normal([n_hid, n_output], stddev=std))
}
b = {
    'bc1': tf.Variable(tf.random_normal([n_filter1], stddev=std)),
    'bc2': tf.Variable(tf.random_normal([n_filter2], stddev=std)),
    'bd1': tf.Variable(tf.random_normal([n_hid], stddev=std)),
    'bd2': tf.Variable(tf.random_normal([n_output], stddev=std))
}
x = tf.placeholder("float", [None, dim]) 
y = tf.placeholder("float", [None, nclass]) 
keepratio = tf.placeholder(tf.float32)

print ("CNN PARAMETERS READY")

CNN PARAMETERS READY


In [4]:
def convolutional_neural_network(_x, _w, _b, _imgsize, _keepratio):
    _input_r = tf.reshape(_x, shape=[-1, _imgsize[0], _imgsize[1], 3])
    # CONVOLUTION LAYER 1
    _conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_input_r
        , _w['wc1'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    # CONVOLUTION LAYER 2
    _conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_pool1
        , _w['wc2'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    _dense1 = tf.reshape(_pool2
                         , [-1, _w['wd1'].get_shape().as_list()[0]])
    # DENSE LAYER 1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # OUTPUT LAYER 2
    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    # RETURN
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1
        , 'conv2': _conv2, 'pool2': _pool2
        , 'dense1': _dense1, 'fc1': _fc1, 'fc_dr1': _fc_dr1, 'out': _out
    }
    return out
print ("CONVOLUTIONAL NEURAL NETWORK MODEL READY")

CONVOLUTIONAL NEURAL NETWORK MODEL READY


# DEFINE FUNCTIONS

In [5]:
learning_rate = 0.001
pred = convolutional_neural_network(x, W, b, imgsize, keepratio)['out']
ce_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
WEIGHT_DECAY_FACTOR = 0.001
l2_loss = tf.add_n([tf.nn.l2_loss(v) 
            for v in tf.trainable_variables()])
cost = ce_cost + WEIGHT_DECAY_FACTOR*l2_loss
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost) 
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, tf.float32))
init = tf.initialize_all_variables()
print ("FUNCTIONS READY")

FUNCTIONS READY


# RUN

In [6]:
# PARAMETERS
batch_size      = 50
display_step    = 20
training_epochs = 200

# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)
    for i in range(num_batch): 
        randidx  = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        feeds    = {x: batch_xs, y: batch_ys, keepratio: 0.6}
        # OPTIMIZE VARIABLES
        sess.run(optm, feed_dict=feeds)
        # COMPUTE LOSS
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch

    # PRINT CURRENT STATUS
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, training_epochs, avg_cost))
        feeds = {x: trainimg, y: trainlabel, keepratio: 1.0}
        train_acc = sess.run(accr, feed_dict=feeds)
        feeds = {x: testimg, y: testlabel, keepratio: 1.0}
        test_acc = sess.run(accr, feed_dict=feeds)
        print (" TRAIN ACCURACY: %.3f" % (train_acc))
        print (" TEST ACCURACY: %.3f" % (test_acc))
        
print ("OPTIMIZATION FINISHED")

Epoch: 000/200 cost: 38.101857567
 TRAIN ACCURACY: 0.792
 TEST ACCURACY: 0.752
Epoch: 020/200 cost: 13.245246410
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.920
Epoch: 040/200 cost: 9.070372868
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.905
Epoch: 060/200 cost: 6.662835693
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.905
Epoch: 080/200 cost: 5.091881227
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.920
Epoch: 100/200 cost: 3.944067883
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.927
Epoch: 120/200 cost: 3.107965708
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.927
Epoch: 140/200 cost: 2.454831719
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.956
Epoch: 160/200 cost: 1.985591149
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.942
Epoch: 180/200 cost: 1.586862469
 TRAIN ACCURACY: 1.000
 TEST ACCURACY: 0.964
OPTIMIZATION FINISHED
